In [34]:
import pandas as pd
import os

In [35]:
current_path = os.getcwd()
dirname, filename = os.path.split(current_path)
print(dirname, filename)

C:\Users\saksh\Documents\20220502_Mlflow_Example notebooks


In [36]:
data_dir= os.path.join(dirname, 'files')
print(data_dir)

C:\Users\saksh\Documents\20220502_Mlflow_Example\files


In [37]:
train_file_path = os.path.join(data_dir, "step1\\train.parquet")

In [38]:
#traindf= pd.read_csv(train_file_path)
traindf= pd.read_parquet(train_file_path)

In [39]:
traindf.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Friend(s),Sunny,80,2PM,Restaurant(20-50),2h,Female,21,Unmarried partner,...,1~3,less1,1~3,never,1,1,0,0,1,0
1,No Urgent Place,Partner,Sunny,80,10AM,Restaurant(20-50),1d,Male,41,Divorced,...,less1,1~3,1~3,1~3,1,0,0,0,1,1
2,Work,Alone,Snowy,30,7AM,Carry out & Take away,1d,Female,50plus,Married partner,...,4~8,less1,1~3,less1,1,1,0,1,0,1
3,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,26,Single,...,1~3,gt8,gt8,gt8,1,1,0,0,1,1
4,Home,Alone,Sunny,80,6PM,Restaurant(<20),1d,Female,50plus,Married partner,...,4~8,less1,1~3,less1,1,0,0,1,0,1


## Analysis Functions

In [40]:
def Columns_to_drop(df):
    #Find out which columns have null values and total number of Nan values in that column
    # Determine percentage of missing data in a particular column for cleaning of the Data. 
    # Columns in data with more than 95% null should be dropped.
    null_count = df.isna().sum()
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_val_columns =list(percent_missing[percent_missing > 95].index)
    return missing_val_columns

In [41]:
def Missing_Val_Columns_to_fill(df):
    #Finding the value counts for columns with percent_missing between 0 and 95%
    #Replacing Nan values in each of these columns with most frequent value in that column.
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_val_col_list_to_fill = list(percent_missing[(percent_missing > 0) & (percent_missing < 95)].index)
    
    return missing_val_col_list_to_fill

## Action Functions

In [42]:
def Drop_Missing_Val_Columns(df, col_list_to_drop):
    df.drop(columns=col_list_to_drop, inplace=True)
    return df

In [43]:
def Fill_missing_Val_Columns(df, col_list_to_fill):
    for i in col_list_to_fill:
        df[i] = df[i].fillna(df[i].value_counts().index[0])
            #print(df[i].isna().sum())
    return df

## Flow

In [44]:
MISSING_VAL_COLUMNS = Columns_to_drop(traindf)
print(MISSING_VAL_COLUMNS)

['car']


In [45]:
MISSING_VAL_COLUMNS_TO_FILL = Missing_Val_Columns_to_fill(traindf)
print(MISSING_VAL_COLUMNS_TO_FILL)

['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']


In [46]:
traindf = Fill_missing_Val_Columns(traindf, MISSING_VAL_COLUMNS_TO_FILL)

In [47]:
traindf = Drop_Missing_Val_Columns(traindf, MISSING_VAL_COLUMNS)

In [48]:
traindf_cleaned= traindf

In [49]:
train_cleaned_file_path = os.path.join(data_dir, "step2\\train_cleaned.parquet")
traindf_cleaned.to_parquet(train_cleaned_file_path, index=False)

## Remarks 

### CONFIG

In [50]:
#MISSING_VAL_COLUMNS = ['car']
#MISSING_VAL_COLUMNS_TO_FILL = ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50']

In [12]:
import yaml
with open("./src/utils/config.yml") as file:
    try:
        databaseConfig = yaml.safe_load(file)   
        print(databaseConfig)
    except yaml.YAMLError as exc:
        print(exc)

{'Categorical_Features': ['destination', 'passanger', 'weather', 'time', 'coupon', 'expiration', 'gender', 'age', 'maritalStatus', 'education', 'occupation', 'income', 'Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50'], 'Numerical_Features': ['temperature', 'has_children', 'toCoupon_GEQ5min', 'toCoupon_GEQ15min', 'toCoupon_GEQ25min', 'direction_same', 'direction_opp', 'Y'], 'MISSING_VAL_COLUMNS': ['car'], 'MISSING_VAL_COLUMNS_TO_FILL': ['Bar', 'CoffeeHouse', 'CarryAway', 'RestaurantLessThan20', 'Restaurant20To50'], 'dict_for_clubbing': {'RestaurantLessThan20': {'1~3': '1~8', '4~8': '1~8'}, 'CarryAway': {'1~3': '1~8ngt8', '4~8': '1~8ngt8', 'gt8': '1~8ngt8', 'less1': 'less1_never', 'never': 'less1_never'}, 'CoffeeHouse': {'1~3': '1~8ngt8', '4~8': '1~8ngt8', 'gt8': '1~8ngt8'}, 'Bar': {'1~3': '1~8', '4~8': '1~8'}, 'income': {'$12500 - $24999': '$12500-$62499', '$25000 - $37499': '$12500-$62499', '$37500 - $49999': '$12500-$62499', '$50000 - $62499': '$12500-$624

In [14]:
databaseConfig['MISSING_VAL_COLUMNS']

['car']

In [11]:
os.chdir('c:\\Users\\saksh\\Documents\\20220502_Mlflow_Example\\')